<a href="https://colab.research.google.com/github/Silsl0/Arapua-Labs/blob/main/ArapuaLabs_AutoDetect_WebMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arapuá Labs — Results & Web Map (Folium)
This notebook reads exported GeoJSON files (wet/dry lagoon polygons and offset vectors), computes KPIs, and builds an interactive Folium map. It includes:
- Google Drive auto-discovery of files and path normalization
- Robust CRS handling (UTM for areas and vector construction)
- Safe use of `union_all()`/`unary_union`
- Vector line creation from `(dx_m, dy_m)` points in meters


In [1]:
%pip install -q folium geopandas shapely pyproj mapclassify

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 15.5 MB/s eta 0:00:00


In [2]:
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

DRIVE_ROOT = '/content/drive/MyDrive'  # Colab default
TARGET_DIR = os.path.join(DRIVE_ROOT, 'ArapuaLabs_GEE_Exports')
os.makedirs(TARGET_DIR, exist_ok=True)
os.makedirs(os.path.join(TARGET_DIR, 'offset_outputs'), exist_ok=True)

WET_DATE = '2024-05-15'
DRY_DATE = '2024-09-15'


Mounted at /content/drive


In [3]:
import glob, shutil, json, os

def find_first(patterns, root):
    for pat in patterns:
        hits = glob.glob(os.path.join(root, '**', pat), recursive=True)
        if hits:
            hits = sorted(hits, key=len)
            return hits[0], hits
    return None, []

wet_path, _ = find_first(['Polygons_Lagoons_Wet.geojson', '*Lagoons*Wet*.geojson'], DRIVE_ROOT)
dry_path, _ = find_first(['Polygons_Lagoons_Dry.geojson', '*Lagoons*Dry*.geojson'], DRIVE_ROOT)
lines_path, _ = find_first(['offset_vectors_lines.geojson', '*offset*lines*.geojson'], DRIVE_ROOT)
points_path, _ = find_first(['offset_vectors.geojson', '*offset*vectors*.geojson'], DRIVE_ROOT)

print('Auto-discovery:')
print('  wet   :', wet_path)
print('  dry   :', dry_path)
print('  lines :', lines_path)
print('  points:', points_path)

def safe_copy(src, dst_dir):
    if src and os.path.exists(src):
        dst = os.path.join(dst_dir, os.path.basename(src))
        if os.path.abspath(src) != os.path.abspath(dst):
            shutil.copy2(src, dst)
        return dst
    return None

if not wet_path or not dry_path:
    raise FileNotFoundError('Wet/Dry lagoon polygons not found in Drive. Ensure they exist and re-run.')

WET_GJ   = safe_copy(wet_path, TARGET_DIR)
DRY_GJ   = safe_copy(dry_path, TARGET_DIR)
LINES_GJ = safe_copy(lines_path, os.path.join(TARGET_DIR, 'offset_outputs')) if lines_path else None
POINTS_GJ= safe_copy(points_path, os.path.join(TARGET_DIR, 'offset_outputs')) if points_path else None

OUT_HTML = os.path.join(TARGET_DIR, 'map_results.html')
OUT_JSON = os.path.join(TARGET_DIR, 'summary_results.json')

autopaths = {
    'DRIVE_DIR': TARGET_DIR,
    'WET_GJ': WET_GJ,
    'DRY_GJ': DRY_GJ,
    'LINES_GJ': LINES_GJ,
    'POINTS_GJ': POINTS_GJ,
    'OUT_HTML': OUT_HTML,
    'OUT_JSON': OUT_JSON,
}
with open(os.path.join(TARGET_DIR, '_autopaths.json'), 'w') as f:
    json.dump(autopaths, f, indent=2)

print('\nNormalized paths:')
for k, v in autopaths.items():
    print(f'{k:10s} = {v}')


Auto-discovery:
  wet   : /content/drive/MyDrive/SertAI_GEE_Exports/Polygons_Lagoons_Wet.geojson
  dry   : /content/drive/MyDrive/SertAI_GEE_Exports/Polygons_Lagoons_Dry.geojson
  lines : None
  points: /content/drive/MyDrive/SertAI_GEE_Exports/offset_outputs/offset_vectors.geojson

Normalized paths:
DRIVE_DIR  = /content/drive/MyDrive/ArapuaLabs_GEE_Exports
WET_GJ     = /content/drive/MyDrive/ArapuaLabs_GEE_Exports/Polygons_Lagoons_Wet.geojson
DRY_GJ     = /content/drive/MyDrive/ArapuaLabs_GEE_Exports/Polygons_Lagoons_Dry.geojson
LINES_GJ   = None
POINTS_GJ  = /content/drive/MyDrive/ArapuaLabs_GEE_Exports/offset_outputs/offset_vectors.geojson
OUT_HTML   = /content/drive/MyDrive/ArapuaLabs_GEE_Exports/map_results.html
OUT_JSON   = /content/drive/MyDrive/ArapuaLabs_GEE_Exports/summary_results.json


In [4]:
import os, json
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString, Point

def safe_exists(p):
    return isinstance(p, (str, os.PathLike)) and len(str(p)) > 0 and os.path.exists(p)

def union_geom_wgs84(gdf):
    g = gdf.to_crs(4326).geometry
    if hasattr(g, 'union_all'):
        return g.union_all()
    return g.unary_union

def auto_utm_crs(gdf):
    u = union_geom_wgs84(gdf)
    c = u.centroid
    lon, lat = float(c.x), float(c.y)
    zone = int((lon + 180)//6) + 1
    south = '+south' if lat < 0 else ''
    return f'+proj=utm +zone={zone} {south} +datum=WGS84 +units=m +no_defs'

def get_center_latlon(gdf):
    u = union_geom_wgs84(gdf)
    c = u.centroid
    return float(c.y), float(c.x)


In [5]:
if not safe_exists(WET_GJ) or not safe_exists(DRY_GJ):
    raise FileNotFoundError('Wet/Dry polygons were not found after normalization.')

wet = gpd.read_file(WET_GJ)
dry = gpd.read_file(DRY_GJ)

utm = auto_utm_crs(wet)
wet_m = wet.to_crs(utm)
dry_m = dry.to_crs(utm)
wet_area_km2 = float(wet_m.area.sum() / 1e6)
dry_area_km2 = float(dry_m.area.sum() / 1e6)
delta_km2    = wet_area_km2 - dry_area_km2

lines = None
if safe_exists(locals().get('LINES_GJ')):
    lines = gpd.read_file(LINES_GJ)
elif safe_exists(locals().get('POINTS_GJ')):
    pts = gpd.read_file(POINTS_GJ)
    utm = auto_utm_crs(wet)
    pts_utm = pts.to_crs(utm)
    F = 5.0
    geoms = []
    rows = []
    if {'dx_m','dy_m'}.issubset(pts.columns):
        for idx, r in pts.iterrows():
            dx = float(r.get('dx_m', np.nan))
            dy = float(r.get('dy_m', np.nan))
            if not (np.isfinite(dx) and np.isfinite(dy)):
                continue
            p = pts_utm.loc[idx].geometry
            if (p is None) or (not isinstance(p, Point)):
                continue
            x, y = float(p.x), float(p.y)
            x2 = x + F*dx
            y2 = y + F*dy
            geoms.append(LineString([(x, y), (x2, y2)]))
            rows.append(idx)
        if geoms:
            keep_cols = [c for c in pts.columns if c in ('speed_m_per_year','dir_deg_math','error','dx_m','dy_m')]
            data = pts.loc[rows, keep_cols].copy()
            lines = gpd.GeoDataFrame(data, geometry=geoms, crs=utm)
        else:
            print('No valid vectors could be built from points (missing dx/dy?).')
            lines = None
    else:
        print("Points file does not have 'dx_m'/'dy_m'. Vectors will be omitted.")

kpis = {}
if (lines is not None) and (not lines.empty) and ('speed_m_per_year' in lines.columns):
    sp = lines['speed_m_per_year'].astype(float).replace([np.inf,-np.inf], np.nan).dropna()
    if len(sp) > 0:
        kpis['speed_p50'] = float(sp.quantile(0.50))
        kpis['speed_p90'] = float(sp.quantile(0.90))
        if {'dx_m','dy_m'}.issubset(lines.columns):
            dx = lines['dx_m'].astype(float); dy = lines['dy_m'].astype(float)
            mean_dir = float(np.degrees(np.arctan2(dy.mean(), dx.mean())))
        elif 'dir_deg_math' in lines.columns:
            mean_dir = float(np.nanmean(lines['dir_deg_math'].astype(float)))
        else:
            mean_dir = np.nan
        mean_bearing = (90 - mean_dir) % 360 if np.isfinite(mean_dir) else np.nan
        kpis['mean_bearing_deg'] = float(mean_bearing) if np.isfinite(mean_bearing) else None

kpis['wet_area_km2'] = wet_area_km2
kpis['dry_area_km2'] = dry_area_km2
kpis['delta_km2']    = delta_km2
kpis['period']       = f"{WET_DATE} → {DRY_DATE}"

with open(OUT_JSON, 'w') as f:
    json.dump(kpis, f, indent=2)

print('KPIs:\n', json.dumps(kpis, indent=2))

center_lat, center_lon = get_center_latlon(wet)


KPIs:
 {
  "speed_p50": 0.0,
  "speed_p90": 29.64876572745471,
  "mean_bearing_deg": 136.89338584574628,
  "wet_area_km2": 623.3762887369749,
  "dry_area_km2": 453.91479669836997,
  "delta_km2": 169.4614920386049,
  "period": "2024-05-15 \u2192 2024-09-15"
}


In [6]:
import folium
from folium.features import GeoJsonTooltip
from folium.plugins import Fullscreen, MiniMap, MeasureControl, MousePosition
import numpy as np, os, json

BINS = [20, 40, 80, 120]
COLS = ['#a6cee3', '#1f78b4', '#33a02c', '#fb9a99', '#e31a1c']
FAST_TH = 20
SIMPLIFY_LINES = True
SIMPL_TOL_M = 5.0

def color_by_speed(v, qs=BINS, cols=COLS):
    try:
        v = float(v)
    except:
        return cols[0]
    for q, c in zip(qs, cols):
        if v <= q:
            return c
    return cols[-1]

def auto_utm_crs(gdf):
    g = gdf.to_crs(4326).geometry
    c = (g.union_all() if hasattr(g, 'union_all') else g.unary_union).centroid
    lon, lat = float(c.x), float(c.y)
    zone = int((lon + 180)//6) + 1
    south = '+south' if lat < 0 else ''
    return f'+proj=utm +zone={zone} {south} +datum=WGS84 +units=m +no_defs'

def bearing_to_compass(b):
    dirs = ['N','NNE','NE','ENE','E','ESE','SE','SSE',
            'S','SSW','SW','WSW','W','WNW','NW','NNW']
    try:
        return dirs[int((float(b) % 360)/22.5 + 0.5) % 16]
    except:
        return '–'

wet_ll = wet.to_crs(4326)
dry_ll = dry.to_crs(4326)

m = folium.Map(location=[center_lat, center_lon], zoom_start=11, tiles='cartodbpositron')
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    name='ESRI World Imagery',
    attr='Esri'
).add_to(m)

folium.GeoJson(
    wet_ll, name='Lagoons — Wet season',
    style_function=lambda f: {
        'color': '#1f78b4', 'weight': 1, 'fillColor': '#1f78b4', 'fillOpacity': 0.15
    }
).add_to(m)

folium.GeoJson(
    dry_ll, name='Lagoons — Dry season',
    style_function=lambda f: {
        'color': '#ffbf00', 'weight': 1, 'fillColor': '#ffbf00', 'fillOpacity': 0.25
    }
).add_to(m)

if (locals().get('lines') is not None) and (not lines.empty):
    L = lines.dropna(subset=['geometry']).copy()
    if SIMPLIFY_LINES and not L.empty:
        try:
            utm = auto_utm_crs(wet)
            L = L.to_crs(utm).copy()
            L['geometry'] = L.geometry.simplify(SIMPL_TOL_M, preserve_topology=True)
            L = L[~L.geometry.is_empty & L.geometry.notna()].copy()
            L = L.to_crs(4326)
        except Exception:
            L = L.to_crs(4326)
    else:
        L = L.to_crs(4326)

    cols_avail = list(getattr(L, 'columns', []))
    tooltip_fields = [c for c in ('speed_m_per_year','dir_deg_math','error') if c in cols_avail]

    def style_fun(feat):
        v = feat['properties'].get('speed_m_per_year', None)
        return {'color': color_by_speed(v), 'weight': 2, 'opacity': 0.9}

    folium.GeoJson(
        L, name='Offset vectors', style_function=style_fun,
        tooltip=GeoJsonTooltip(fields=tooltip_fields,
                               aliases=['speed (m/yr)','dir (deg)','error'][:len(tooltip_fields)])
    ).add_to(m)

    if 'speed_m_per_year' in cols_avail:
        try:
            Lfast = L[L['speed_m_per_year'].astype(float) > FAST_TH]
        except Exception:
            Lfast = L.iloc[0:0]
        if not Lfast.empty:
            folium.GeoJson(
                Lfast, name=f'Vectors (> {FAST_TH} m/yr)',
                style_function=lambda f: {'color':'#e31a1c','weight':2,'opacity':0.9},
                tooltip=GeoJsonTooltip(fields=tooltip_fields,
                                       aliases=['speed (m/yr)','dir (deg)','error'][:len(tooltip_fields)])
            ).add_to(m)

Fullscreen().add_to(m)
MiniMap(toggle_display=True).add_to(m)
m.add_child(MeasureControl(
    position='bottomleft',
    primary_length_unit='kilometers',
    secondary_length_unit='meters',
    primary_area_unit='sqkilometers',
    secondary_area_unit='hectares',
    active_color='#e67e22',
    completed_color='#2c3e50'
))
MousePosition(prefix='lat/lon').add_to(m)
folium.LayerControl(collapsed=False, position='bottomleft').add_to(m)

BASE_CARD_CSS = "background:white;padding:12px 14px;border:1px solid #bbb;font-size:14px;line-height:1.35;"
TITLE_CSS = "font-weight:600;font-size:14px;"

bins_labels = [f'≤{q:.0f}' for q in BINS] + ['>']
legend_html = f'<div style="{BASE_CARD_CSS}">'
legend_html += f'<div style="{TITLE_CSS}">Dune migration speed (m/yr)</div>'
for label, c in zip(bins_labels, COLS + [COLS[-1]]):
    legend_html += f'<div><i style="background:{c};width:12px;height:12px;display:inline-block;margin-right:8px;border:1px solid #888;"></i>{label}</div>'
legend_html += '</div>'

k = {}
try:
    with open(OUT_JSON, 'r') as f:
        k = json.load(f)
except Exception:
    k = {}

def bearing_to_compass(b):
    dirs = ['N','NNE','NE','ENE','E','ESE','SE','SSE','S','SSW','SW','WSW','W','WNW','NW','NNW']
    try:
        return dirs[int((float(b) % 360)/22.5 + 0.5) % 16]
    except:
        return '–'

bearing = k.get('mean_bearing_deg', None)
bearing_str = bearing_to_compass(bearing) if bearing is not None else '–'

card_html = f"""
    <div style="{BASE_CARD_CSS}">
      <div style="{TITLE_CSS}">Arapuá Labs — Results (SAR / AOT)</div>
      <div>Period: {k.get('period','–')}</div>
      <hr style=\"margin: 6px 0; border-top: 1px solid #ddd;\">
      <div style=\"{TITLE_CSS}; color:#1f78b4;\">Hydrology (Lagoon)</div>
      <div>Lagoon area — wet season: {k.get('wet_area_km2',0):,.1f} km²</div>
      <div>Lagoon area — dry season: {k.get('dry_area_km2',0):,.1f} km²</div>
      <div style=\"font-weight:600;\">Δ Lagoon Area: {k.get('delta_km2',0):+,.1f} km²</div>
      <hr style=\"margin: 6px 0; border-top: 1px solid #ddd;\">
      <div style=\"{TITLE_CSS}; color:#e31a1c;\">Aeolian (Dune Migration)</div>
      <div>Migration Speed (90% of Active Dunes): <span style=\"font-weight:600;\">{k.get('speed_p90',0):,.0f} m/yr</span></div>
      <div>Overall Motion Speed (Median): {k.get('speed_p50',0):,.0f} m/yr</div>
      <div>Mean Migration Direction: <span style=\"font-weight:600;\">{bearing_str}</span></div>
    </div>"""

ui_stack = f"""
<div style=\"position: fixed; top: 20px; right: 20px; z-index: 9999; display: flex; gap: 14px;\">
  {card_html if card_html else ''}
  {legend_html}
</div>
"""
m.get_root().html.add_child(folium.Element(ui_stack))

m.save(OUT_HTML)
OUT_HTML

'/content/drive/MyDrive/ArapuaLabs_GEE_Exports/map_results.html'

In [7]:
from IPython.display import IFrame
IFrame(OUT_HTML, width=1100, height=650)